# **ML Pipeline: Predict Sale Price**

## Objectives

* Fit and evaluate a regression model that will predict the Sale Price of a house in Ames, Iowa
* Create an ML Pipeline including data cleaning, feature engineering and ML model steps

## Inputs

* outputs/datasets/collection/HousePriceRecords.csv
* Data Cleaning and Feature Engineering steps found in previous notebooks

## Outputs

* TrainSet
* TestSet
* Regression Piepline
* Feature importance Plot

## CRISP-DM

* Modelling
* Evaluation


---

# Change working directory

* The notebooks are stored in a sub folder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

In [ ]:
import warnings

# Load Data

In [ ]:
import pandas as pd
df = pd.read_csv(f"outputs/datasets/collection/HousePriceRecords.csv")
df.head(5)

### Load variables

* Below we will declare the relevant variables as decided upon on the Data Cleaning and Feature Engineering notebooks.

* Data Cleaning

In [ ]:
arbitrary_imputation_vars = ['2ndFlrSF', 'EnclosedPorch', 'WoodDeckSF']
median_imputation_vars = ['BedroomAbvGr', 'LotFrontage', 'GarageYrBlt',
                          'MasVnrArea']
most_frequent_vars = ['BsmtFinType1', 'GarageFinish']

* Categorical Encoding

In [ ]:
categorical_encoding_vars = (df.select_dtypes(include=['object'])
                             .columns.to_list())
categorical_encoding_vars

* Feature Engineering

In [ ]:
log_transformation_vars = ['1stFlrSF', 'LotArea']
power_vars = ['2ndFlrSF', 'BsmtFinSF1', 'BsmtUnfSF',
              'GarageArea', 'MasVnrArea', 'TotalBsmtSF']
yeojohnson_vars = ['GrLivArea']

# Create ML Pipeline
* We create the base for the ML Pipeline
* We inlcude the steps for Data Cleaning Feature Engineering, Feature Scaling, Feature Selection and the ML model we decide upon
* We will use Hyperparameter Optimization to choose the best model and most effective hyperparamters

In [ ]:
from sklearn.pipeline import Pipeline

from feature_engine.selection import DropFeatures
from feature_engine.imputation import (MeanMedianImputer,
                                       ArbitraryNumberImputer,
                                       CategoricalImputer)

# Feature Engineering
from feature_engine.selection import SmartCorrelatedSelection
from feature_engine.encoding import OrdinalEncoder
from feature_engine import transformation as vt
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from feature_engine.outliers import Winsorizer

warnings.filterwarnings('ignore')


def PipelineRegressor(model):
    pipeline_base = Pipeline([

        ("ArbitraryImputer",
         ArbitraryNumberImputer(arbitrary_number=0,
                                variables=arbitrary_imputation_vars)),

        ("MedianImputation",
         MeanMedianImputer(imputation_method='median',
                           variables=median_imputation_vars)),

        ("CategoricalImputer",
         CategoricalImputer(imputation_method='frequent',
                            variables=most_frequent_vars)),

        # Feature Engineering
        ("OrdinalCategoricalEncoder",
         OrdinalEncoder(encoding_method='arbitrary',
                        variables=categorical_encoding_vars)),

        ("LogTransformer", vt.LogTransformer(
            variables=['1stFlrSF', 'LotArea'])),

        ("PowerTransformer", vt.PowerTransformer(variables=power_vars)),

        ("YeoJohnsonTransformer", vt.YeoJohnsonTransformer(
            variables=yeojohnson_vars)),

        ("Winsorizer", Winsorizer(capping_method='iqr', tail='both', fold=1.5,
                                  variables=['GrLivArea'])),

        ("SmartCorrelatedSelection",
         SmartCorrelatedSelection(variables=None, method="spearman",
                                  threshold=0.75,
                                  selection_method="variance")),

        # Feature Scaling
        ("scaler", StandardScaler()),

        # Feature Selection
        ("feat_selection", SelectFromModel(model)),

        # ML Algorithms
        ("model", model),
    ])

    return pipeline_base

* **Hyperparameter Optimization Search**
    * Below code taken from CI lesson: *Scikit-Learn Unit 6: Cross Validation Search Part 2*

In [ ]:
from sklearn.model_selection import GridSearchCV


class HyperparameterOptimizationSearch:

    def __init__(self, models, params):
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}

    def fit(self, X, y, cv, n_jobs, verbose=1, scoring=None, refit=False):
        for key in self.keys:
            print(f"\nRunning GridSearchCV for {key} \n")

            model = PipelineRegressor(self.models[key])
            params = self.params[key]
            gs = GridSearchCV(model, params, cv=cv, n_jobs=n_jobs,
                              verbose=verbose, scoring=scoring, )
            gs.fit(X, y)
            self.grid_searches[key] = gs

    def score_summary(self, sort_by='mean_score'):
        def row(key, scores, params):
            d = {
                 'estimator': key,
                 'min_score': min(scores),
                 'max_score': max(scores),
                 'mean_score': np.mean(scores),
                 'std_score': np.std(scores),
            }
            return pd.Series({**params, **d})

        rows = []
        for k in self.grid_searches:
            params = self.grid_searches[k].cv_results_['params']
            scores = []
            for i in range(self.grid_searches[k].cv):
                key = "split{}_test_score".format(i)
                r = self.grid_searches[k].cv_results_[key]
                scores.append(r.reshape(len(params), 1))

            all_scores = np.hstack(scores)
            for p, s in zip(params, all_scores):
                rows.append((row(k, s, p)))

        df = pd.concat(rows, axis=1).T.sort_values([sort_by], ascending=False)
        columns = ['estimator', 'min_score', 'mean_score', 'max_score',
                   'std_score']
        columns = columns + [c for c in df.columns if c not in columns]
        return df[columns], self.grid_searches
        

# Split Train & Test Set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
                                    df.drop(['SalePrice'], axis=1),
                                    df['SalePrice'],
                                    test_size=0.2,
                                    random_state=0,
                                    )

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

* We import our potential Algorithms

In [ ]:
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression

# Scikit Learn Cross Validation Search

* We first search for the most suitable algorithm using tandard hyper parameters

In [ ]:
models_quick_search = {
    "LogisticRegression": LogisticRegression(random_state=0),
    "LinearRegression": LinearRegression(),
    "GradientBoostingRegressor": GradientBoostingRegressor(random_state=0),
    "XGBRegressor": XGBRegressor(random_state=0),
    "DecisionTreeRegressor": DecisionTreeRegressor(random_state=0),
    "RandomForestRegressor": RandomForestRegressor(random_state=0),
    "ExtraTreesRegressor": ExtraTreesRegressor(random_state=0),
    "AdaBoostRegressor": AdaBoostRegressor(random_state=0),
}

params_quick_search = {
    "LogisticRegression": {},
    "LinearRegression": {},
    "GradientBoostingRegressor": {},
    "XGBRegressor": {},
    "DecisionTreeRegressor": {},
    "RandomForestRegressor": {},
    "GradientBoostingClassifier": {},
    "ExtraTreesRegressor": {},
    "AdaBoostRegressor": {},
}

* We will include LogisticRegression in our first run. For this the max cross validation value we can have is cv=2.
* We will check how LogisticRegression performs against the other algorithms. If it is not the best performer, we will remove it from our search and run again with a greater cross validation value.

In [ ]:
from sklearn.metrics import make_scorer, r2_score

search = HyperparameterOptimizationSearch(models=models_quick_search,
                                          params=params_quick_search)
search.fit(X_train, y_train, scoring='r2', n_jobs=-1, cv=2)

* We add the results to a DataFrame and assess.

In [ ]:
import numpy as np
grid_search_summary,
grid_search_pipelines = search.score_summary(sort_by='mean_score')

print(grid_search_summary.shape)
grid_search_summary

* LogisticRegression had the worst overall mean R2 score so we can remove it and run again with corss valdation value of cv=5
* We redefine our variables without LogisticRegression and complete another search

In [ ]:
models_quick_search = {
    "LinearRegression": LinearRegression(),
    "GradientBoostingRegressor": GradientBoostingRegressor(random_state=0),
    "XGBRegressor": XGBRegressor(random_state=0),
    "DecisionTreeRegressor": DecisionTreeRegressor(random_state=0),
    "RandomForestRegressor": RandomForestRegressor(random_state=0),
    "ExtraTreesRegressor": ExtraTreesRegressor(random_state=0),
    "AdaBoostRegressor": AdaBoostRegressor(random_state=0),
}

params_quick_search = {
    "LinearRegression": {},
    "GradientBoostingRegressor": {},
    "XGBRegressor": {},
    "DecisionTreeRegressor": {},
    "RandomForestRegressor": {},
    "GradientBoostingClassifier": {},
    "ExtraTreesRegressor": {},
    "AdaBoostRegressor": {},
}

In [ ]:
search = HyperparameterOptimizationSearch(models=models_quick_search,
                                          params=params_quick_search)
search.fit(X_train, y_train, scoring='r2', n_jobs=-1, cv=5)

* We check how the algorithms performed

In [ ]:
grid_search_summary,
grid_search_pipelines = search.score_summary(sort_by='mean_score')

print(grid_search_summary.shape)
grid_search_summary

* LinearRegression provided us with our best mean score of 0.78.
* This score is sufficient to meet the performance goal of our model as per our client's business case (R2 score > 0.75)
* We can see that GradientBoostingRegressor also provides us with a suffiecient R2 score so we may consider this also 
* Next we will try find the best hyperparameters for our models and try to fine tune them so that we can improve the scores on our TrainSet

In [ ]:
models_search = {
    "ExtraTreesRegressor": ExtraTreesRegressor(random_state=0),
}

# }
# params_search = {
#     "ExtraTreesRegressor": {
#         'model__max_depth': [3,5,10],
#         'model__min_impurity_decrease': [0.0,0.3,0.5,1],
#         'model__min_samples_leaf': [0.1,0.5,1],
#         'model__min_samples_split': [2,3,5],
#         'model__min_weight_fraction_leaf': [0.0,0.1,0.5],
#         'model__n_estimators': [100,300,400]
#     }
# }
params_search = {
    "ExtraTreesRegressor": {
        'model__max_depth': [10],
        'model__min_impurity_decrease': [0.0],
        'model__min_samples_leaf': [1],
        'model__min_samples_split': [5],
        'model__min_weight_fraction_leaf': [0.0],
        'model__n_estimators': [100, 300]}
}

* We perform another Cross Validation search using "LinearRegression" & "GradientBoostingRegressor" as our model and a set of hyperparameters to apply, in order to find the best hyperparameter permutation for our model

In [ ]:
search = HyperparameterOptimizationSearch(models=models_search,
                                          params=params_search)
search.fit(X_train, y_train, scoring='r2', n_jobs=-1, cv=5)

* We add the results to a DataFrame and assess

In [ ]:
grid_search_summary,
grid_search_pipelines = search.score_summary(sort_by='mean_score')

grid_search_summary.head()

* We can see that we have an improvement in the mean R2 score, bringing it up to 0.81
* We will use these hyperparameters for our model

* We print our best model below

In [ ]:
best_model = grid_search_summary.iloc[0, 0]
best_model

* We print our best combination of hyperparameters for our algorithm

In [ ]:
best_hyperparams = grid_search_pipelines[best_model].best_params_
best_hyperparams

* We define our best pipeline at this stage

In [ ]:
pipeline_regressor = grid_search_pipelines[best_model].best_estimator_
pipeline_regressor

# Measure performance on Train & Test Sets

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid")


def regression_performance(X_train, y_train, X_test, y_test, pipeline):
    print("Model Evaluation \n")
    print("* Train Set")
    regression_evaluation(X_train, y_train, pipeline)
    print("* Test Set")
    regression_evaluation(X_test, y_test, pipeline)


def regression_evaluation(X, y, pipeline):
    prediction = pipeline.predict(X)
    print('R2 Score:', r2_score(y, prediction).round(3))
    print('Mean Absolute Error:', mean_absolute_error(y, prediction).round(3))
    print('Mean Squared Error:', mean_squared_error(y, prediction).round(3))
    print('Root Mean Squared Error:',
          np.sqrt(mean_squared_error(y, prediction)).round(3))
    print("\n")


def regression_evaluation_plots(X_train, y_train, X_test, y_test, pipeline,
                                alpha_scatter=0.5):
    pred_train = pipeline.predict(X_train)
    pred_test = pipeline.predict(X_test)

    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(18, 12))
    sns.scatterplot(x=y_train, y=pred_train, alpha=alpha_scatter, ax=axes[0])
    sns.lineplot(x=y_train, y=y_train, color='red', ax=axes[0])
    axes[0].set_xlabel("Actual")
    axes[0].set_ylabel("Predictions")
    axes[0].set_title("Train Set")

    sns.scatterplot(x=y_test, y=pred_test, alpha=alpha_scatter, ax=axes[1])
    sns.lineplot(x=y_test, y=y_test, color='red', ax=axes[1])
    axes[1].set_xlabel("Actual")
    axes[1].set_ylabel("Predictions")
    axes[1].set_title("Test Set")

    plt.show()

In [ ]:
regression_performance(X_train, y_train, X_test, y_test, pipeline_regressor)
regression_evaluation_plots(X_train, y_train, X_test, y_test,
                            pipeline_regressor)

* The R2 score on our test set is 0.782.
* This is an acceptable score, in the context of our business case, for our house price prediction model.

# Assess feature importance

* We are now interetsed to check which are the most important features for our pipeline and as our algorithm is Tree Based, we can access these using the 'feature_importances_' method.
    * The below code was taken from CI walthrough project: *Churnometer*

In [ ]:
%matplotlib inline
sns.set_style('whitegrid')

# number of data cleaning and feature engineering in pipeline
data_cleaning_feat_eng_steps = 9
columns_after_data_cleaning_feat_eng = ((Pipeline(pipeline_regressor
                                        .steps[:data_cleaning_feat_eng_steps])
                                        .transform(X_train)
                                        .columns))

best_features = (columns_after_data_cleaning_feat_eng[pipeline_regressor
                 ['feat_selection'].get_support()].to_list())

# # create DataFrame to display feature importance
df_feature_importance = (pd.DataFrame(data={
          'Feature': (columns_after_data_cleaning_feat_eng
                      [pipeline_regressor['feat_selection'].get_support()]),
          'Importance': pipeline_regressor['model'].feature_importances_})
  .sort_values(by='Importance', ascending=False)
  )

# # Most important features statement and plot
print(f"* These are the {len(best_features)} most important features in" +
      f"descending order. "
      f"The model was trained on them:" +
      f"\n{df_feature_importance['Feature'].to_list()}")

df_feature_importance.plot(kind='bar', x='Feature', y='Importance')
plt.show()

* The most important features for predicting 'SalePrice' will be printed out below

In [ ]:
best_features

# Rewrite & Refit Pipeline with best features
* We rewrite our pipeline removing all steps whihc do not include our most important features.
* We also remove SmartCorelatedSelectio and Feature Selection. As we have our most important features, these are no longer necessary.
* We aim to streamline our pipeline to predict 'SalePrice' using fewer features.

In [ ]:
def PipelineRegressor(model):
    pipeline_base = Pipeline([
        ("PowerTransformer", vt.PowerTransformer(variables=['GarageArea',
                                                            'TotalBsmtSF'])),

        ("YeoJohnsonTransformer",
         vt.YeoJohnsonTransformer(variables=yeojohnson_vars)),

        ("Winsorizer", Winsorizer(capping_method='iqr', tail='both', fold=1.5,
                                  variables=['GrLivArea'])),
        ("scaler", StandardScaler()),
        ("model", model),
    ])

    return pipeline_base

# Evaluating on Train and Test Sets

* We split our Train & Test Sets again using only our most important features

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
                                    df.drop(['SalePrice'], axis=1),
                                    df['SalePrice'],
                                    test_size=0.2,
                                    random_state=0
                                    )

print("* Train set:", X_train.shape, y_train.shape, "\n* Test set:",
      X_test.shape, y_test.shape)

In [ ]:
X_train = X_train.filter(best_features)
X_test = X_test.filter(best_features)

print("* Train set:", X_train.shape, y_train.shape, "\n* Test set:",
      X_test.shape, y_test.shape)
X_train.head(3)

# Grid Search

* We define our highest performing model and hyperparameters

In [ ]:
models_search = {
    "ExtraTreesRegressor": ExtraTreesRegressor(random_state=0)
}

params_search = {
    "ExtraTreesRegressor": {
        'model__max_depth': [10],
        'model__min_impurity_decrease': [0.0],
        'model__min_samples_leaf': [1],
        'model__min_samples_split': [5],
        'model__min_weight_fraction_leaf': [0.0],
        'model__n_estimators': [100]}

}

params_search


* We assess the performance on the scaled down TrainSets

In [ ]:
search = HyperparameterOptimizationSearch(models=models_search,
                                          params=params_search)
search.fit(X_train, y_train, scoring='r2', n_jobs=-1, cv=5)

In [ ]:
grid_search_summary,
grid_search_pipelines = search.score_summary(sort_by='mean_score')

grid_search_summary

* Our R2 score remains at 0.81

* We define our best model and pipeline

In [ ]:
best_model = grid_search_summary.iloc[0, 0]
best_model

In [ ]:
pipeline_regressor = grid_search_pipelines[best_model].best_estimator_
pipeline_regressor

In [ ]:
# number of data cleaning and feature engineering in pipeline
data_cleaning_feat_eng_steps = 3
columns_after_data_cleaning_feat_eng = ((Pipeline(pipeline_regressor
                                        .steps[:data_cleaning_feat_eng_steps])
                                        .transform(X_train)
                                        .columns))

best_features = (columns_after_data_cleaning_feat_eng[pipeline_regressor
                 ['feat_selection'].get_support()].to_list())

# # create DataFrame to display feature importance
df_feature_importance = (pd.DataFrame(data={
          'Feature': (columns_after_data_cleaning_feat_eng
                      [pipeline_regressor['feat_selection'].get_support()]),
          'Importance': pipeline_regressor['model'].feature_importances_})
  .sort_values(by='Importance', ascending=False)
  )

# # Most important features statement and plot
print(f"* These are the {len(best_features)} most important features in" +
      f"descending order. "
      f"The model was trained on them:" +
      f"\n{df_feature_importance['Feature'].to_list()}")

df_feature_importance.plot(kind='bar', x='Feature', y='Importance')
plt.show()

* The feature importance remains the same

* We assess the performance on our TestSet

In [ ]:
regression_performance(X_train, y_train, X_test, y_test, pipeline_regressor)
regression_evaluation_plots(X_train, y_train, X_test, y_test,
                            pipeline_regressor)

* The R2 score remains at 0.782.

---

# Push files to Repo

We will generate the following files for our dashboard:

* Train set
* Test set
* Modelling Pipeline
* Fetaure Importance Plot

In [ ]:
import joblib
import os

version = 'v2'
file_path = f"outputs/ml_pipeline/predict_saleprice/{version}"
try:
    os.makedirs(name=file_path)
except Exception as e:
    print(e)

### Train Set: features and target

In [ ]:
X_train.head(3)

In [ ]:
X_train.to_csv(f"{file_path}/X_train.csv", index=False)


### Test Set: features and target

In [ ]:
X_test.head(3)

In [ ]:
X_test.to_csv(f"{file_path}/X_test.csv", index=False)

In [ ]:
y_train

In [ ]:
y_train.to_csv(f"{file_path}/y_train.csv", index=False)

In [ ]:
y_test

In [ ]:
y_test.to_csv(f"{file_path}/y_test.csv", index=False)

### Modelling Pipeline

* ML pipeline for SalePrice prediction

In [ ]:
pipeline_regressor

In [ ]:
joblib.dump(value=pipeline_regressor,
            filename=f"{file_path}/pipeline_regressor.pkl")

### Feature importance plot

In [ ]:
df_feature_importance.plot(kind='bar', x="Feature", y="Importance")
plt.savefig(f"{file_path}/features_importance.png", bbox_inches="tight",
            facecolor='white')